# **Some examples of Reinforcement Learning problems**

# I. A Multi-Armed Bandit

We will now look at a practical example of a Reinforcement Learning problem - the multi-armed bandit problem (one of the most popular problems in RL).

> *You are faced repeatedly with a choice among k different options, or actions. After each choice you receive a numerical reward chosen from a stationary probability distribution that depends on the action you selected. Your objective is to maximize the expected total reward over some time period, for example, over 1000 action selections, or time steps.*

You can think of it in analogy to a slot machine (a one-armed bandit). Each action selection is like a play of one of the slot machine’s levers, and the rewards are the payoffs for hitting the jackpot.

**Solving this problem means that we can come up with an optimal policy: a strategy that allows us to select the best possible action (the one with the highest expected return) at each time step.**


## I. 1. Action-Value Methods
A very simple solution is based on the action value function. Remember that an action value is the mean reward when that action is selected:
$$ q(a)= \mathbb{E} (R_t∣A = a)$$
We can easily estimate q using the sample average:

$$
Q_t(a) = \frac{\mbox{sum of rewards when } "a" \mbox{ taken prior to }"t"}
{\mbox{number of times } "a" \mbox{ taken prior to } "t"}
$$

If we collect enough observations, our estimate gets close enough to the real function. We can then act greedily at each timestep, i.e. select the action with the highest value, to collect the highest possible rewards.

## I.2.  Exploration vs. exploitation

As a matter of fact, if we always act greedily as proposed in the previous paragraph, we never try out sub-optimal actions which might actually eventually lead to better results.

To introduce some degree of exploration in our solution, we can use an $\varepsilon $-greedy strategy: we select actions greedily most of the time, but every once in a while, with probability $\varepsilon$, we select a random action, regardless of the action values.

It turns out that this simple exploration method works very well, and it can significantly increase the rewards we get.

One final caveat - to avoid from making our solution too computationally expensive, we compute the average incrementally according to this formula:
$$
Q_{n+1} = Q_n + \frac{1}{n}[R_n−Q_n]
$$

In [1]:
import numpy as np

# Number of bandits
k = 3

# True probability of winning for each bandit
p_bandits = [0.45, 0.40, 0.56]

def pull(a):
    """Pull arm of bandit with index `i` and return 1 if win,
    else return 0."""
    if np.random.rand() < p_bandits[a]:
        return 1
    else:
        return 0


In [2]:
T = 1000
# Our action values
Q = [0 for _ in range(k)]

# This is to keep track of the number of times we take each action
N = [0 for _ in range(k)]

eps = 0.8

suma = 0
for x in range(T):
    if np.random.rand() > eps:
        # Take greedy action most of the time
        a = np.argmax(Q)
    else:
        # Take random action with probability eps
        a = np.random.randint(0, k)

    # Collect reward
    reward = pull(a)
    suma += reward

    # Incremental average
    N[a] += 1
    Q[a] += 1/N[a] * (reward - Q[a])

print(Q)
print(suma)

[0.41218637992831525, 0.37956204379562025, 0.5659955257270696]
472


If we run this script for a couple of seconds, we already see that our action values are proportional to the probability of hitting the jackpots for our bandits.


# II. Self-driving Taxi problem
Imagine we're teaching a taxi how to transport people in a parking lot to four different locations (R,G,Y,B) .

You can setup up the taxi-problem environment using OpenAi’s Gym, which is one of the most used libraries for solving reinforcement problems. To install the library, use the Python package installer (pip):

In [3]:
pip install gym

In [4]:
!pip install cmake 'gym[atari]' scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.6 MB/s eta 0:00:00


Now let’s see how our environment is going to render. All the models and interface for this problem are already configured in Gym and named under Taxi-V3. Use the code snippet below to render this environment:

> <font color='blue'> “There are 4 locations (labelled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.”</font> (Source: https://gym.openai.com/envs/Taxi-v3/ )

This will be the rendered output on your console:


    "+---------+",
    "|R: | : :G|",
    "| : | : : |",
    "| : : : : |",
    "| | : | : |",
    "|Y| : |B: |",
    "+---------+",



Env is the core of OpenAi Gym, which is the unified environment interface. The following are the env methods that will be quite helpful to us:

1.   **<font color='red'>env.reset</font>**: Resets the
environment and returns a random initial state.

2.  **<font color='red'> env.step(action)</font>**: Step the environment by one timestep. It returns the following variables:

> * <font color='red'> observation</font>: Observations of the environment.
* <font color='red'> reward</font>: If your action was beneficial or not.
* <font color='red'> done</font>: Indicates if we have successfully picked up and dropped off a passenger, also called one episode.
* <font color='red'> info</font>: Additional info such as performance and latency for debugging purposes.

3. **<font color='red'> env.render</font>**: Renders one frame of the environment (helpful in visualizing the environment).


In [5]:
import gym

env = gym.make("Taxi-v3").env


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [6]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Action Space Discrete(6)
State Space Discrete(500)


## II.1. State Space


Lets dive deeper in breaking down the problem. The taxi is the only car in this parking lot. We can break the parking lot into a 5x5 grid, which gives us 25 possible taxi locations. These 25 locations are one part of our state space. Notice the current location state of our taxi is coordinate (3, 1).

![axi-v2 Env](https://storage.googleapis.com/lds-media/images/Reinforcement_Learning_Taxi_Env.width-1200.png)

In the environment, there are four possible locations where you can drop off the passengers: R, G, Y, B or [(0,0), (0,4), (4,0), (4,3)] in (row, col) coordinates if you can interpret the above-rendered environment as a coordinate axis.

When we also account for one (1) additional passenger state of being inside the taxi, we can take all combinations of passenger locations and destination locations to come to a total number of states for our taxi environment — there are four (4) destinations and five (4 + 1) passenger locations. So, our taxi environment has 5×5×5×4=500 total possible states.



In [7]:
state = env.encode(3, 1, 2, 0)
# (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328


## II.2 Action Space

The agent encounters one of the 500 states, and it takes action. The action in our case can be to move in a direction or decide to pick up/drop off a passenger.

In other words, we have six possible actions:  **<font color='red'>pickup, drop, north, east,south, west</font>**
(These four directions are the moves by which the taxi is moved.)

This is the action space: the set of all the actions that our agent can take in a given state.

You’ll notice in the illustration above, that the taxi cannot perform certain actions in certain states due to walls. In the environment’s code, we will simply provide a -1 penalty for every wall hit and the taxi won’t move anywhere. This will just rack up penalties causing the taxi to consider going around the wall.



## II.3 Reward Table:

When the taxi environment is created, there is an initial reward table that’s also created, called P. We can think of it like a matrix that has the number of states as rows and number of actions as columns, i.e. states × actions matrix.

Since every state is in this matrix, we can see the default reward values assigned to our illustration’s state:

In [8]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

This dictionary has a structure

<font color='green'>{action: [(probability, nextstate, reward, done)]}.</font>

> * The 0–5 corresponds to the actions (south, north, east, west, pickup, drop off) the taxi can perform at our current state in the illustration.
  - 0 = south
  - 1 = north
  - 2 = east
  - 3 = west
  - 4 = pickup
  - 5 = dropof
* done is used to tell us when we have successfully dropped off a passenger in the right location.


## II.4. Solution without RL

Let's see what would happen if we try to brute-force our way to solving the problem without RL.

Since we have our P table for default rewards in each state, we can try to have our taxi navigate just using that.

We'll create an infinite loop which runs until one passenger reaches one destination (one episode), or in other words, when the received reward is 20. The env.action_space.sample() method automatically selects one random action from set of all possible actions.

In [9]:
# set environment to illustration's state
env.s = 328

# Setting the number of iterations, penalties and reward to zero,
epochs = 0
penalties, reward = 0, 0

# for animation
frames = []

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1

    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'episode': 0,
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1


print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Timesteps taken: 7382
Penalties incurred: 2414


### Printing frames

In [10]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Episode: {frame['episode']}")
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode: 0
Timestep: 7382
State: 410
Action: 5
Reward: 20


Not good. Our taxi takes thousands of timesteps and makes lots of wrong drop offs to deliver just one passenger to the right destination.

This is because we aren't learning from past experience. We can run this over and over, and it will never optimize. The agent has no memory of which action was best for each state, which is exactly what Reinforcement Learning will do for us.

## II.5. Reinforcement Learning using Q-Learning

Essentially, Q-learning lets the agent use the environment's rewards to learn, over time, the best action to take in a given state.

In our Taxi environment, we have the reward table, P, that the agent will learn from. It does thing by looking receiving a reward for taking an action in the current state, then updating a Q-value to remember if that action was beneficial.

The values store in the Q-table are called a Q-values, and they map to a (state, action) combination.

A Q-value for a particular state-action combination is representative of the "quality" of an action taken from that state. Better Q-values imply better chances of getting greater rewards.

For example, if the taxi is faced with a state that includes a passenger at its current location, it is highly likely that the Q-value for pickup is higher when compared to other actions, like dropoff or north.

Q-values are initialized to an arbitrary value, and as the agent exposes itself to the environment and receives different rewards by executing different actions, the Q-values are updated using the equation:
$$
Q({\small state}, {\small action}) \leftarrow (1 - \alpha) Q({\small state}, {\small action}) + \alpha \Big({\small reward} + \gamma \max_{a} Q({\small next \ state}, {\small all \ actions})\Big)
$$
Where:

- $\alpha$ (alpha) is the learning rate ($0< \alpha \leq 1$) - Just like in supervised learning settings, $\alpha$  is the extent to which our Q-values are being updated in every iteration.

- $\gamma$ (gamma) is the discount factor ($0\leq \gamma \leq 1$) - determines how much importance we want to give to future rewards. A high value for the discount factor (close to 1) captures the long-term effective award, whereas, a discount factor of 0 makes our agent consider only immediate reward, hence making it greedy.

In [11]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

**Q-Table**

The Q-table is a matrix where we have a row for every state (500) and a column for every action (6). It's first initialized to 0, and then values are updated after training. Note that the Q-table has the same dimensions as the reward table, but it has a completely different purpose.

![alt text](https://storage.googleapis.com/lds-media/images/q-matrix-initialized-to-learned_gQq0BFs.width-1200.png)


### Below is the algorithm in brief:

* Step 1: Initialize the Q-table with all zeros and Q-values to arbitrary constants.
* Step 2: Let the agent react to the environment and explore the actions. For each change in state, select any one among all possible actions for the current state (S).
* Step 3: Travel to the next state (S’) as a result of that action (a).
* Step 4: For all possible actions from the state (S’) select the one with the highest Q-value.
* Step 5: Update Q-table values using the equation.
* State 6: Change the next state as the current state.
* Step 7: If goal state is reached, then end and repeat the process.

In [12]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 20s, sys: 6.92 s, total: 1min 27s
Wall time: 1min 26s


Check the q_table

In [13]:
q_table[382]

array([-2.46770886, -2.4510224 , -2.46413162, -2.4510224 , -9.81996992,
       -9.67518131])

### Evaluate smart taxi performance after Q-learning

In [14]:
total_epochs, total_penalties = 0, 0
episodes = 100
frames = []

for ep in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'episode': ep,
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.77
Average penalties per episode: 0.0


### Visualization of Q-learning

In [15]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Episode: 99
Timestep: 1377
State: 475
Action: 5
Reward: 20


# Acknowledgment
The materials were prepared on the base of following materials:



https://builtin.com/data-science/reinforcement-learning-python

https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/


https://www.kaggle.com/karthikcs1/reinforcement-learning-taxi-v3-openai

https://stackabuse.com/introduction-to-reinforcement-learning-with-python/

https://github.com/casey-barr/open-ai-taxi-problem/blob/master/open_ai_taxi_problem.ipynb